In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
X = df.drop("sales", axis=1)
y = df["sales"]

In [5]:
from sklearn.preprocessing import PolynomialFeatures

In [6]:
# создаем конвертер
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [7]:
# создаем новые признаки
poly_features = polynomial_converter.fit_transform(X)

In [8]:
poly_features.shape

(200, 19)

In [9]:
# разбиение данных на наборы + масштабирование признаков
from sklearn.model_selection import train_test_split

In [10]:
# вместо X укажен poly_features чтобы использовать 19 признаков
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [11]:
# проверим X_train 
X_train.shape

(140, 19)

In [12]:
# масштабирование признаков
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [14]:
# далее для этого объекта scaler нужно вычислить метрики ср значения и СКО
# чтобы не допустить утечки данных мы будем вычислять метрики только на обучающем наборе данных
scaler.fit(X_train)

StandardScaler()

In [15]:
X_train = scaler.transform(X_train)

In [16]:
X_test = scaler.transform(X_test)

In [17]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [18]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [19]:
from sklearn.linear_model import Ridge

In [20]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)
 |
 |  Linear least squares with l2 regularization.
 |
 |  Minimizes the objective function::
 |
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.0
 |      Constant that multiplies the L2 term, controlling regularization
 | 

In [21]:
ridge_model = Ridge(alpha=10)

In [22]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [23]:
test_predictions = ridge_model.predict(X_test)

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [25]:
MAE = mean_absolute_error(y_test, test_predictions)
print(f'MAE: {MAE}')

MAE: 0.5774404204714183


In [26]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
print(f'RMSE: {RMSE}')

RMSE: 0.8946386461319676


In [27]:
from sklearn.linear_model import RidgeCV # CV = Cross Validation

In [28]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10))  # Эти значения по умолчанию, их можно было бы и не указывать. Указывать, если нужно отличное от них

In [29]:
ridge_cv_model.fit(X_train, y_train) # Из-за метода здесь уже не только обучающий сет, но и валидационный сет, т.к. выбирается значение альфа

RidgeCV(alphas=(0.1, 1.0, 10))

In [30]:
ridge_cv_model.alpha_ # Какое альфа было наилучшим?

0.1

In [31]:
# Чтобы понять, по каким метрикам выбирается наилучшее альфа - можно посмотреть их все
from sklearn.metrics._scorer import _SCORERS
_SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro',

In [32]:
# Пробуем с метрикой MAE
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10), scoring='neg_mean_absolute_error')

In [33]:
ridge_cv_model.fit(X_train, y_train) # Из-за метода здесь уже не только обучающий сет, но и валидационный сет, т.к. выбирается значение альфа

RidgeCV(alphas=(0.1, 1.0, 10), scoring='neg_mean_absolute_error')

In [34]:
ridge_cv_model.alpha_ # Какое альфа было наилучшим?

0.1

In [35]:
# Вычисляем метрики

In [36]:
test_predictions = ridge_cv_model.predict(X_test)

In [37]:
# При этом метрики были вычислиены на y_test, те это hold out test
MAE = mean_absolute_error(y_test, test_predictions) 
print(f'MAE: {MAE}')

RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
print(f'RMSE: {RMSE}')
# Ошибка меньше, мы нашли оптимальное значение с помощью кросс валидации

MAE: 0.42737748843287143
RMSE: 0.6180719926935899


In [38]:
# Смотрим коэффы. Для L2 регуляризации коэффы не равны нулю
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [39]:
ridge_cv_model.best_score_
# Это наилучшее значение метрики - в нашем случае это метрика MAE (neg_min_absolute_error выбирали ранее)

-0.3749223340292957

In [40]:
from sklearn.linear_model import LassoCV
# Сразу берем вариант с кросс валидацией
## Можно передавать значения альфа. Либо списков как в ридж, либо оставить как есть (те none). Тогда будут работать параметры эпсилон и n-alphas. 
### Похоже на linspace. С помощью параметра n-alphas мы указываем кол-во параметров альфа, в том диапазоне, кот. определяется параметром эпсилон. При этом эпсилон - это соотношение между минимальным и максимальным значением альфа. По сути это такой способ указать ширину диапазона значений альфа. При этом сами значения sklearn определяет автоматически

In [41]:
# При всем при этом можно использовать значения по умолчанию
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5, max_iter=1000000)
# чем больше n, тем естественно больше времени займут вычисления. чем меньше eps, тем больший диапазон будем обрабатывать
## cv - кол-во частей, на кот. разобем. По умолчанию 5.
## иногда вызыв модели в следующем действии вызывает ошибку несходимости. тогда увеличиваем max_iter с дефолтных 1000. еще можно уменьшить eps

In [42]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1, max_iter=1000000)

In [43]:
lasso_cv_model.alpha_

0.4943070909225828

In [44]:
test_predictions = lasso_cv_model.predict(X_test)

In [45]:
MAE = mean_absolute_error(y_test, test_predictions) 
print(f'MAE: {MAE}')

RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
print(f'RMSE: {RMSE}')

MAE: 0.6541723161252858
RMSE: 1.1308001022762542


In [46]:
lasso_cv_model.coef_
# Преимущество - почти все коэффы равны нулю кроме двух

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [47]:
from sklearn.linear_model import ElasticNetCV

In [48]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.001, n_alphas=100, max_iter=1000000)

In [49]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [50]:
# посмотрим на аттрибут l1 ratio - какой модель выбрала наилучшим значением
elastic_model.l1_ratio_

1.0

In [51]:
# посмотрим на альфа. если сравним с альфой в модели lasso cv, то получим то же значение альфа
elastic_model.alpha_

0.004943070909225827

In [52]:
test_predictions = elastic_model.predict(X_test)

In [53]:
MAE = mean_absolute_error(y_test, test_predictions) 
print(f'MAE: {MAE}')

RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
print(f'RMSE: {RMSE}')

MAE: 0.4335034618590072
RMSE: 0.6063140748984036
